In [ ]:
#Preamble stuff, loading up libraries and convenience functions
import all_data_handler
import pandas
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
import itertools
import seaborn as sb
import scipy.stats as ss #For ss.f_oneway() ANOVAs

import re

#Given the object list of a gesture from the coding, return a list of letters representing normalized 
#values for the objects of the gesture
def tag_object(original):
    #This use of strip is to prevent quotes from messing up regex matches
    original = " ".join(event["objects"]).strip("\"")
	tags = []
	#Matches robots, robot, bot, bots, etc. 
	robots = re.compile("bot|group|swarm|orange|red", re.I)
	crate = re.compile("crate", re.I)
	targetA = re.compile("area a|box a| a$|to a,|^a$", re.I)
	targetB = re.compile("area b|box b| b$|to b,|^b$", re.I)
	whitespace = re.compile("whitespace|ground|screen|white area", re.I)
	
	toCheck = [(robots, "r"), (crate, "c"), (targetA, "a"), (targetB, "b"), (whitespace, "w")]
	for compiled, tag in toCheck:
		if re.search(compiled, original):
			tags.append(tag)
	return tags

def count_select_taps(participant):
    #Dict of task id to count of taps on robots
    counts = {}
    for task_id in participant["tasks"].keys():
        #Default to no counts of select taps
        counts[task_id] = 0
        events = participant["tasks"][task_id]
        #Flip through all the events
        for event in events:
            #If we have objects, tag them
            if "objects" in event.keys():
                tags = tag_object(event["objects"])
                #We have both an event and a set of tags, check if it's a tap on a robot
                if 'r' in tags and event["event_type"] == "tap":
                    counts[task_id] += 1
    return counts

def count_group_select(participant):
    #Dict of task id to count of taps on robots
    counts = {}
    for task_id in participant["tasks"].keys():
        #Default to no counts of select taps
        counts[task_id] = 0
        events = participant["tasks"][task_id]
        #Flip through all the events
        for event in events:
            #If we have objects, tag them
            if "objects" in event.keys():
                tags = tag_object(event["objects"])
                #We have both an event and a set of tags, check if it's a tap on a robot
                if 'r' in tags and (event["event_type"] == "lasso" or event["event_type"] == "box_select"):
                    counts[task_id] += 1
    return counts

def count_box_select(participant):
    #Dict of task id to count of taps on robots
    counts = {}
    for task_id in participant["tasks"].keys():
        #Default to no counts of select taps
        counts[task_id] = 0
        events = participant["tasks"][task_id]
        #Flip through all the events
        for event in events:
            #If we have objects, tag them
            if "objects" in event.keys():
                tags = tag_object(event["objects"])
                #We have both an event and a set of tags, check if it's a tap on a robot
                if 'r' in tags and event["event_type"] == "box_select":
                    counts[task_id] += 1
    return counts

def count_lasso_select(participant):
    #Dict of task id to count of taps on robots
    counts = {}
    for task_id in participant["tasks"].keys():
        #Default to no counts of select taps
        counts[task_id] = 0
        events = participant["tasks"][task_id]
        #Flip through all the events
        for event in events:
            #If we have objects, tag them
            if "objects" in event.keys():
                tags = tag_object(event["objects"])
                #We have both an event and a set of tags, check if it's a tap on a robot
                if 'r' in tags and event["event_type"] == "lasso":
                    counts[task_id] += 1
    return counts

#Returns a dict of task IDs to count of total events
def count_events(particpant):
counts = {}
    for task_id in participant["tasks"].keys():
        #Default to no counts of select taps
        counts[task_id] = 0
        #Flip through all the events
        for event in events:
            if event["event_type"] == "memo":
                #This event is a memo, not a user gesture
                continue
            elif example in event.keys() and event["example"] == True:
                #This event is an example, don't count it
                continue
            else:
                #Not one of the skip cases, so count it
                counts[task_id] += 1
    
#This gets a dict of user ids to dicts of task ids to select tap counts
all_select_taps = adh.apply(count_select_taps)
all_group_selects = adh.apply(count_group_select)
all_lassos = adh.apply(count_lasso_select)
all_box = adh.apply(count_box_select)
all_gesture_counts = adh.apply(count_events)

#Prints out the per-condition counts, one entry for each user in that condition
#Also gets the mean and standard deviation for the condition
def user_counts(user_counts):
    per_cond = {}
    #For each user, maintain a running total
    for pid in user_counts.keys():
        #Get their condition
        cond = adh.IdToCondition(pid)[0]
        counts = user_counts[pid].values()
        per_user_total = sum(counts)
        if cond in per_cond.keys():
            per_cond[cond].append(per_user_total)
        else:
            per_cond[cond] = [per_user_total]
    
    for cond in per_cond.keys():
        
        total = sum(per_cond[cond])
        mean = np.mean(per_cond[cond])
        std_dev = np.std(per_cond[cond])
        print cond, per_cond[cond]
        print "total:{} mean:{} std dev:{}".format(total, mean, std_dev)
        print

#Given a dict of user IDs to dicts of task ids to event counts
#return a dict of user IDs to lists of event counts for the common tasks
def get_matched_tasks(counts):
    #Get a list of the task names that every condition has in common
    common_names = []
    for name in adh.taskMap.keys():
        if all(adh.taskMap[name].values()):
            common_names.append(name)
    common_names.sort()

    #dict of user ids to a list of counts, index of counts is task number
    matched_tasks = {}
    for user in counts.keys():
        common_counts = []
        for task in common_names:
            #print user, adh.IdToCondition(user), task, adh.taskNameToNumber(task, user)
            #Get the count at the task number for this common name
            common_counts.append(counts[user][str(adh.taskNameToNumber(task, user))])
        matched_tasks[user] = common_counts
    return matched_tasks

#Given a dict of conditions to lists of samples
#Do an all-pairs 1-way ANOVA on the samples
def all_pairs_f(d):
    for k1, k2 in itertools.combinations(d.keys(), 2):
        print k1, k2
        print ss.f_oneway(d[k1], d[k2])
        print

H1: There exists a number of robots beyond which users will transition from treating robots as individuals to interacting with the robots in small groups or as a single large group. 

This transition point will be apparent because of a change in the gesture set that the user uses to interact with the swarm. It is hypothesized that above the transition point, users will be more likely to neglect some subset of the available robots. The user will instead use commands that control the bulk of the robots as a cloud or flock, but may leave some robots unused. For example, the user may switch from selecting robots as individuals to shaping and pushing the swarm the way a child might play with a bug, putting their hand down so the bug goes around or avoids it, touching the back of the bug gently to make it scurry forwards, and so forth, or by shaping the group as if sculpting, with pushing and pinching to "carry" groups around. 

The user may also change how they indicate which robots are to be interacted with. Rather than selecting each robot by clicking on it, the may "paint" over the area containing the robots they want to use, or draw a circle around them. 
The size of the swarm where changes in the user gestures occur will indicate the transition point between interacting with individual robots and interacting with the swarm as a whole. 

This hypothesis would be invalidated by the gestures selected by the user displaying no correlation with the size of the swarm that they are controlling.

H2: A display which obscures individual robots and displays a cloud or swarm boundary will cause the user to treat the swarm as a whole rather than individuals, which will be apparent because the user will use the same gestures they would use to control a single robot. 

Once the ratio of the size of individual swarm members to the size of the area the swarm is in becomes sufficiently large, displaying the swarm members at the same scale as the map will result in the representation of the swarm members being too small to interact with. This problem will arise at smaller scales if the swarm robots are themselves quite tiny, and some of the available swarm robots are indeed small. Scaling the representation of the robots up, relative to the map, will make the robot representations overlap unrealistically and obscure the map. Instead, we propose that for certain scales of swarms, it makes sense to represent the swarm as the area covered, rather than the locations of the individual robots. This approach has been used successfully for navigation in three dimensions, by developing a controller that causes the individual UAVs to remain within a bounding prism, and allowing the user to control the shape and location of that prism.

Altering how the user interface displays the location of the robots in the swarm will affect the transition point. 

This hypothesis would be invalidated by the gestures selected by the user in the single robot case being dissimilar from those selected in the case where the swarm is displayed as a cloud or covered region. However, some variation is expected in commands for tasks which a swarm can do, which a single robot cannot perform, such as dividing into two groups. 
